<a href="https://colab.research.google.com/github/felladib/H_SentimentAnalysis_REC/blob/main/SA_withRES%26NORM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment model**

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Concatenate, LayerNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
import numpy as np

In [ ]:
class AddNorm(tf.keras.layers.Layer):
    """Residual connection followed by layer normalization."""

    def __init__(self, name=None, **kwargs):
        super().__init__()
        if name: self._name = name
        self.ln = LayerNormalization(**kwargs)
        self.add = Add()

    def call(self, X):
        return self.ln(self.add(X))

In [ ]:
from keras.backend import dropout
class HybLstm(Model):
  def __init__(self,units,preprocessing_model,l1=0,l2=0):
    super().__init__()
    self.preprocessing_model = preprocessing_model
    self.first_layer = Dense(units,activation='relu')
    self.lstm1 = Bidirectional(LSTM(int(units/2), return_sequences=True, kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2) , dropout = 0.5))
    self.lstm2 = Bidirectional(LSTM(int(units/2), return_sequences=True, kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2) , dropout = 0.5))
    self.lstm3 = LSTM(units, return_sequences=True, return_state=True, kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2) , dropout = 0.5)

    self.add1,self.add2 = AddNorm(),AddNorm()

    self.concat = Concatenate()
    self.dense1 = Dense(units,activation='relu')
    self.final_layer = Dense(1,activation="sigmoid")

  def call(self,x):
    x  = self.preprocessing_model(x)
    x  = self.first_layer(x)
    xl = self.lstm1(x)
    x  = self.add1([x,xl])
    xl = self.lstm2(x)
    x  = self.add2([x,xl])

    _, final_memory_state, final_carry_state  = self.lstm3(x)
    states = self.concat([final_memory_state, final_carry_state])
    states = self.dense1(states)
    out    = self.final_layer(states)
    return out

  def train(self,X_train,Y_train, batch_size , epochs, learning_rate,**kwargs):
    print("    training ...")
    # Learning rate decay
    callbacks = []
    def scheduler(epoch, lr):
      if epoch<2:
        return learning_rate/10
      else:
        return learning_rate * np.exp(-4 * epoch / epochs)

    callbacks.append(tf.keras.callbacks.LearningRateScheduler(scheduler))

    # Compile the model :
    self.compile(loss='binary_crossentropy' ,
                 optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
                 metrics = ["accuracy" , rmse , 'mae' , precision])
    # Fitting the data :
    self.train_history = self.fit(X_train,Y_train,
                                  batch_size = BATCH_SIZE,
                                  epochs=EPOCHS,
                                  callbacks=callbacks,
                                  ** kwargs)

In [ ]:
BATCH_SIZE = 32
EPOCHS = 15
LEARNING_RATE = 5e-3

In [ ]:
model = HybLstm(units=32,preprocessing_model=preprocessing_model,l1=1e-5,l2=1e-5)

In [ ]:
history = model.train(X_train_rev,y_train_rev, batch_size = BATCH_SIZE, epochs=EPOCHS, learning_rate=LEARNING_RATE , validation_data=(X_val_rev , y_val_rev))

    training ...
Epoch 1/15
1250/1250 [==============================] - 581s 455ms/step - loss: 0.4644 - accuracy: 0.7822 - root_mean_squared_error: 0.3508 - mae: 0.2905 - precision: 0.8189 - val_loss: 0.4924 - val_accuracy: 0.7864 - val_root_mean_squared_error: 0.3938 - val_mae: 0.2458 - val_precision: 0.9238 - lr: 5.0000e-04
Epoch 2/15
1250/1250 [==============================] - 563s 451ms/step - loss: 0.3612 - accuracy: 0.8468 - root_mean_squared_error: 0.3294 - mae: 0.2171 - precision: 0.8440 - val_loss: 0.3459 - val_accuracy: 0.8572 - val_root_mean_squared_error: 0.3224 - val_mae: 0.2165 - val_precision: 0.8644 - lr: 5.0000e-04
Epoch 3/15
1250/1250 [==============================] - 563s 451ms/step - loss: 0.3559 - accuracy: 0.8512 - root_mean_squared_error: 0.3268 - mae: 0.2138 - precision: 0.8503 - val_loss: 0.3530 - val_accuracy: 0.8536 - val_root_mean_squared_error: 0.3267 - val_mae: 0.1961 - val_precision: 0.8148 - lr: 0.0029
Epoch 4/15
1250/1250 [==========================

In [ ]:
model.evaluate(X_test_rev , y_test_rev)

157/157 [==============================] - 59s 374ms/step - loss: 0.5945 - accuracy: 0.8592 - root_mean_squared_error: 0.3496 - mae: 0.1451 - precision: 0.8547


[0.5945040583610535,
 0.8592000007629395,
 0.34960469603538513,
 0.14513042569160461,
 0.8546671867370605]

overfitting





In [ ]:
from keras import metrics
rmse = metrics.RootMeanSquaredError()
precision = metrics.Precision()